In [24]:

import pandas as pd
from load_data import map_tf_ids, separate_tf_genes, prepare_for_inference
import db as db
from utils_network import *

In [25]:
config_file = "/data_nfs/og86asub/alternet-project/alternet/configs/MAGNet_NF.yaml"
import yaml

with open(config_file, 'r') as f:
    config = yaml.safe_load(f)


# Load necessary data and annotations

In [26]:
biomart_path = '/data_nfs/og86asub/alternet-project/alternet/data/biomart.txt'
tf_list_path = '/data_nfs/og86asub/alternet-project/alternet/data/allTFs_hg38.txt'

biomart = pd.read_csv(biomart_path, sep='\t')
tf_list = pd.read_csv(tf_list_path, sep='\t', header = None)
tf_list = map_tf_ids(tf_list, biomart)

# create TF Database
tf_database = db.create_transcipt_annotation_database(tf_list=tf_list, appris_path= config['appris'], digger_path=config['digger'])


/data_nfs/og86asub/alternet-project/alternet/src/alternet/db.py:32: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  digger = pd.read_csv(digger_path)


In [27]:
tf_database

,TF,Gene stable ID,Transcript stable ID,Gene name (HGNC),Translation ID,Is translated?,Transcript type,Not found tag,CCDS ID,Transcript support level,...,Trans-membrane helices (THUMP),Signal sequence (CRASH),Trifid Score,Peptides,APPRIS score,APPRIS Annotation,MANE,Exon stable ID,Exon rank in transcript,Pfam ID
0,A1CF,ENSG00000148584,ENST00000373997,A1CF,ENSP00000363109,TRANSLATION,protein_coding,-,CCDS7241.1,1,...,0.0,"-4,-5",1.0000,29,12.000,ALTERNATIVE:1,MANE_Select,"[ENSE00003539976, ENSE00003688232, ENSE0000368...","[3, 4, 4, 5, 5, 6, 6, 6, 7, 7, 8, 8, 9, 9, 10,...","[PF00076, PF00076, PF00076, PF00076, PF00076, ..."
1,A1CF,ENSG00000148584,ENST00000374001,A1CF,ENSP00000363113,TRANSLATION,protein_coding,-,CCDS7241.1,5,...,0.0,"-4,-5",0.9986,29,12.000,ALTERNATIVE:1,NaN,"[ENSE00001345632, ENSE00003539976, ENSE0000368...","[1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 6, 7, 7, 8, 8, ...","[PF00076, PF00076, PF00076, PF00076, PF00076, ..."
2,A1CF,ENSG00000148584,ENST00000414883,A1CF,ENSP00000397953,TRANSLATION,protein_coding,-,CCDS7241.1,5,...,0.0,"-4,-5",0.9373,29,12.000,ALTERNATIVE:1,NaN,"[ENSE00001345632, ENSE00003539976, ENSE0000368...","[1, 4, 5, 5, 6, 6, 7, 7, 2, 3]","[PF00076, PF00076, PF00076, PF00076]"
3,A1CF,ENSG00000148584,ENST00000373993,A1CF,ENSP00000363105,TRANSLATION,protein_coding,-,CCDS7242.1,1,...,0.0,"-4,-5",0.9733,30,11.508,PRINCIPAL:3,NaN,"[ENSE00003539976, ENSE00003688232, ENSE0000368...","[1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 7, 7, 9, ...","[PF00076, PF00076, PF00076, PF00076, PF00076, ..."
4,A1CF,ENSG00000148584,ENST00000373995,A1CF,ENSP00000363107,TRANSLATION,protein_coding,-,CCDS7243.1,2,...,0.0,"-4,-5",0.2064,27,6.998,MINOR,NaN,"[ENSE00001345632, ENSE00003688232, ENSE0000368...","[1, 4, 4, 5, 5, 6, 6, 6, 7, 7, 8, 8, 9, 9, 10,...","[PF00076, PF00076, PF00076, PF00076, PF00076, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16293,ZXDB,ENSG00000198455,ENST00000374888,ZXDB,ENSP00000364023,TRANSLATION,protein_coding,-,CCDS35313.1,-,...,0.0,"-4,-5",0.8479,-,13.000,PRINCIPAL:1,MANE_Select,"[ENSE00001465020, ENSE00001465020, ENSE0000146...","[1, 1, 1, 1, 1]","[PF00096, PF00096, PF00096, PF00096, PF00096]"
16294,ZXDC,ENSG00000070476,ENST00000514463,ZXDC,-,NO_TRANSLATION,retained_intron,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16295,ZXDC,ENSG00000070476,ENST00000389709,ZXDC,ENSP00000374359,TRANSLATION,protein_coding,-,CCDS43145.1,1,...,0.0,"-4,-5",1.0000,1,15.000,PRINCIPAL:2,MANE_Select,"[ENSE00001506699, ENSE00001506699, ENSE0000150...","[1, 1, 1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]","[PF00096, PF00096, PF00096]"
16296,ZXDC,ENSG00000070476,ENST00000515545,ZXDC,ENSP00000426532,TRANSLATION,nonsense_mediated_decay,start,-,1,...,0.0,"-4,-5",0.0398,1,-2.000,MINOR,NaN,NaN,NaN,NaN


In [28]:
## LOAD gene and transcript data
transcript_data = pd.read_csv(config['transcript_data'], index_col=0)

## GET the gene ids associated with transcripts
gene_ids = transcript_data['gene_id'].unique()




In [29]:
import rnanorm

In [30]:
## REad the gene data
gene_data = pd.read_csv(config['count_data'], index_col=0)
gene_data.index.name = 'gene_id'
gene_data = gene_data.reset_index()
gene_data = gene_data[gene_data['gene_id'].isin(gene_ids)]

gene_data = gene_data.set_index('gene_id')


In [31]:
gene_data = gene_data.iloc[0:20:, ]

In [32]:
fpkm = rnanorm.FPKM('/data_nfs/datasets/GRCh38/Homo_sapiens.GRCh38.109.gtf').set_output(transform="pandas")
gene_data = fpkm.fit_transform(gene_data.T)

In [33]:
gene_data= gene_data.T

In [34]:
gene_data.index.name = 'gene_id'
gene_data = gene_data.reset_index()

In [35]:

## Subset the samples of interest
sample_attributes = pd.read_csv(config['sample_attributes'])
sample_attributes = sample_attributes.loc[:, ['sample_name', 'etiology']]
samples = sample_attributes[sample_attributes['etiology'] == config['tissue']]
samples = samples['sample_name'].tolist()


## Get unified gene and transcript table
gene_data = gene_data.loc[:, ['gene_id'] + samples ]
transcript_data = transcript_data.loc[:,['gene_id', 'transcript_id'] + samples]


In [36]:
transcript_data

,gene_id,transcript_id,C00039,C00055,C00074,C00085,C00105,C00120,C00132,C00158,...,P01600,P01603,P01610,P01616,P01621,P01622,P01623,P01626,P01628,P01635
0,ENSG00000244682,ENST00000473530,2.276150,2.040930,1.564230,2.629880,3.224940,0.081621,3.577190,3.275500,...,1.130160,0.495594,1.458520,2.579210,0.110632,1.342680,2.268880,2.249030,0.000000,2.658050
1,ENSG00000244682,ENST00000467903,2.167410,2.788660,3.829120,3.177140,4.211690,0.176553,6.959880,3.485350,...,2.218410,1.574970,2.190090,2.435190,0.364871,2.658690,2.882860,4.777670,0.256739,2.298990
2,ENSG00000244682,ENST00000508651,1.950990,2.569730,3.989870,3.134920,5.018950,0.110165,4.027630,1.973540,...,3.231000,1.006310,0.281987,2.741400,0.052092,2.602130,3.153650,4.473280,0.185469,1.629250
3,ENSG00000280938,ENST00000626647,0.049179,0.019637,0.017334,0.489663,0.000000,0.105393,0.517216,0.029199,...,0.234445,0.072602,0.436067,0.057276,0.373421,0.103772,0.383861,0.494979,6.911700,0.000000
4,ENSG00000282181,ENST00000633377,3.231750,1.134710,3.935840,0.548367,0.007815,0.319113,3.020600,1.936820,...,0.478180,0.111222,0.650404,0.288542,0.006630,1.255300,1.448720,2.497850,0.007706,1.090220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39310,ENSG00000197081,ENST00000569097,0.821758,0.377072,0.413925,0.907076,0.476571,0.098923,0.395161,0.379031,...,0.434826,0.292465,0.449118,0.594457,0.060498,0.269001,0.681752,0.957061,0.047140,0.631274
39311,ENSG00000225366,ENST00000602699,0.050290,0.014330,0.035171,0.041823,0.037060,0.000000,0.065104,0.016308,...,0.051768,0.040899,0.000000,0.036091,0.000000,0.022102,0.044203,0.044029,0.000000,0.051400
39312,ENSG00000182986,ENST00000594408,2.056060,1.068780,0.909087,1.211280,1.235250,0.022664,1.194140,1.571050,...,2.038440,0.777621,0.761086,1.157570,0.064273,0.759715,1.245030,1.833670,0.017389,1.783930
39313,ENSG00000205482,ENST00000510091,0.397195,0.139399,0.137387,0.238294,0.456423,0.055239,0.127514,0.330749,...,0.067772,0.278306,0.285876,0.041657,0.033892,0.187877,0.468567,1.001040,0.049527,0.233459


In [37]:
biomart

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Gene name,Transcript name,Gene type
0,ENSG00000210049,ENSG00000210049.1,ENST00000387314,ENST00000387314.1,MT-TF,MT-TF-201,Mt_tRNA
1,ENSG00000211459,ENSG00000211459.2,ENST00000389680,ENST00000389680.2,MT-RNR1,MT-RNR1-201,Mt_rRNA
2,ENSG00000210077,ENSG00000210077.1,ENST00000387342,ENST00000387342.1,MT-TV,MT-TV-201,Mt_tRNA
3,ENSG00000210082,ENSG00000210082.2,ENST00000387347,ENST00000387347.2,MT-RNR2,MT-RNR2-201,Mt_rRNA
4,ENSG00000209082,ENSG00000209082.1,ENST00000386347,ENST00000386347.1,MT-TL1,MT-TL1-201,Mt_tRNA
...,...,...,...,...,...,...,...
278215,ENSG00000228437,ENSG00000228437.5,ENST00000441160,ENST00000441160.1,LINC02474,LINC02474-202,lncRNA
278216,ENSG00000229463,ENSG00000229463.2,ENST00000412098,ENST00000412098.2,LYST-AS1,LYST-AS1-201,lncRNA
278217,ENSG00000229291,ENSG00000229291.2,ENST00000438371,ENST00000438371.1,LINC02768,LINC02768-201,lncRNA
278218,ENSG00000229291,ENSG00000229291.2,ENST00000715678,ENST00000715678.1,LINC02768,LINC02768-203,lncRNA


In [38]:
tf_iso_counts, _ = separate_tf_genes(data=transcript_data, tf_list=tf_list, data_column='transcript_id', biomart_column='Transcript stable ID')

In [39]:
tf_gene_counts, target_gene_counts = separate_tf_genes(data=gene_data, tf_list=tf_list, data_column='gene_id', biomart_column='Gene stable ID')

In [40]:
data_canonical, data_asware, target_gene_list = prepare_for_inference(tf_iso_counts, tf_gene_counts, target_gene_counts, transcript_column='transcript_id', gene_column='gene_id')

Preparing data for inference: Separation into canonical and as-aware gene expression data


In [41]:
# get isoform categories
isoform_categories = isoform_categorization(tf_iso_counts, tf_gene_counts, threshold_dominance=20)


In [42]:
from inference import *

In [43]:
as_aware_grn, canonical_grn = inference(config, 1, tf_list=tf_list, data_canonical=data_canonical, data_asware=data_asware, target_gene_list=target_gene_list, aggregate=True)


Starting inference ...


/data_nfs/og86asub/alternet-project/alternet/.pixi/envs/default/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33487 instead
  warnings.warn(


1

Computing network


PermissionError: [Errno 13] Permission denied: '/data_nfs/mi34qaba/SpliceAwareGRN/results/NF/grn/NF_as-aware.network_1.tsv'